<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_2_%E1%84%89%E1%85%A1%E1%84%82%E1%85%A2_%E1%84%80%E1%85%B2%E1%84%8C%E1%85%A5%E1%86%BC_%E1%84%8E%E1%85%A2%E1%86%BA%E1%84%87%E1%85%A9%E1%86%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사내 규정 챗봇 만들기



## RAG 실습 준비

In [ ]:
!pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 75.3 MB/s eta 0:00:00
 

In [ ]:
import os

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# ==========================================
# 0. 초기 설정
# ==========================================
import os
from getpass import getpass

# OpenAI API key
openai_api_key = getpass("Enter your OpenAI API key: ")

# API키 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API key: ··········


In [ ]:
# 실습용 가상 데이터 생성 (사내 규정 문서)
# 실제로는 PDF나 txt 파일이 있는 경우가 많지만, 실습 편의를 위해 즉석에서 파일을 만듭니다.
policy_content = """
[주식회사 랭체인 인사 규정]
제1조 (근무 시간)
1. 근무 시간은 오전 9시부터 오후 6시까지로 한다.
2. 유연 근무제를 시행하며, 코어 타임(오전 11시~오후 3시)을 준수해야 한다.

제2조 (휴가)
1. 연차 휴가는 입사 1년 후 15일이 발생한다.
2. 3년 이상 근속 시 안식월 1개월을 유급으로 제공한다.
3. 반차, 반반차 사용이 가능하며 당일 신청도 가능하다.
4. 여름 휴가는 7월~8월 중 5일을 별도로 제공한다.

제3조 (복지)
1. 도서 구입비는 월 10만원 한도 내에서 무제한 지원한다.
2. 야근 시 택시비와 식대는 법인 카드로 결제한다.
"""

# 파일을 저장합니다.
with open("company_policy.txt", "w", encoding="utf-8") as f:
    f.write(policy_content)

print("[준비 완료] 사내 규정 문서가 생성되었습니다.")

[준비 완료] 사내 규정 문서가 생성되었습니다.


## RAG 실습 시작 - LangChain 사용

### 문서 로딩 (Loading)

In [ ]:
# ==========================================
# 1. 문서 로딩 (Loading)
# ==========================================
print("\n 문서를 불러옵니다...")
loader = TextLoader("company_policy.txt", encoding="utf-8")
docs = loader.load()
print(f"   -> 문서 길이: {len(docs[0].page_content)}자")


 문서를 불러옵니다...
   -> 문서 길이: 321자


### 문서 분할 (Splitting)

In [ ]:
# ==========================================
# 2. 문서 분할 (Splitting)
# ==========================================
print("\n 문서를 작은 조각(Chunk)으로 나눕니다...")
# chunk_size=100: 100자 단위로 자름
# chunk_overlap=20: 문맥이 끊기지 않도록 20자는 겹치게 함
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)
print(f"   -> 총 {len(splits)}개의 조각(Chunk)으로 분할되었습니다.")


 문서를 작은 조각(Chunk)으로 나눕니다...
   -> 총 5개의 조각(Chunk)으로 분할되었습니다.


### 임베딩 & 벡터 저장소 생성 (Indexing)

In [ ]:
# ==========================================
# 3. 임베딩 & 벡터 저장소 생성 (Indexing)
# ==========================================
print("\n 문서를 벡터로 변환하여 저장합니다...")
# OpenAI의 가성비 임베딩 모델 사용
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)
print("   -> Chroma DB에 저장 완료!")


 문서를 벡터로 변환하여 저장합니다...
   -> Chroma DB에 저장 완료!


### 검색기(Retriever) 설정 & 생성 (Generation)

In [ ]:
# ==========================================
# 4. 검색기(Retriever) 설정 & 생성 (Generation)
# ==========================================
print("\n RAG 체인을 생성합니다...")

# 검색기 설정 (가장 유사한 문서 3개를 뽑아오도록 설정)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


 RAG 체인을 생성합니다...


In [ ]:
# 프롬프트 템플릿 작성
# {context}: 검색된 문서 내용이 들어갈 자리
# {question}: 사용자 질문이 들어갈 자리
template = """
당신은 회사의 인사 담당 AI입니다.
아래 [규정]을 참고하여 직원의 질문에 친절하게 답해주세요.
규정에 없는 내용은 "규정에 나와있지 않습니다"라고 답하세요.

[규정]
{context}

[질문]: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
# 체인 연결 (Retrieval -> Prompt -> LLM -> Parser): LangChain Expression Language(LCEL)
# RunnablePassthrough:질문 그대로 쓰고 싶은 경우, RunnableLambda:질문을 가공하고 싶을 경우
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### 실행 (Test)

In [ ]:
# ==========================================
# 5. 실행 (Test)
# ==========================================
question = "입사한 지 3년 되면 어떤 혜택이 있어? 그리고 책 사는 건 얼마나 지원해줘?"

print(f"\n 질문: {question}")
print("AI 답변 생성 중...\n")

response = rag_chain.invoke(question)

print("="*30)
print(response)
print("="*30)


 질문: 입사한 지 3년 되면 어떤 혜택이 있어? 그리고 책 사는 건 얼마나 지원해줘?
AI 답변 생성 중...

입사한 지 3년이 지나면 안식월 1개월을 유급으로 제공받게 됩니다. 또한, 도서 구입비는 월 10만원 한도 내에서 무제한 지원됩니다.


### 마무리 정리

In [ ]:
# 실습 종료 후 임시 파일 삭제
os.remove("company_policy.txt")